In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

        
df_train = pd.read_csv('/kaggle/input/amexpert2019/train.csv')
df_demo = pd.read_csv('/kaggle/input/amexpert2019/customer_demographics.csv')
df_coup = pd.read_csv('/kaggle/input/amexpert2019/coupon_item_mapping.csv')
df_cust = pd.read_csv('/kaggle/input/amexpert2019/customer_transaction_data.csv')
df_camp = pd.read_csv('/kaggle/input/amexpert2019/campaign_data.csv')
df_item_data = pd.read_csv('/kaggle/input/amexpert2019/item_data.csv')
df_test = pd.read_csv('/kaggle/input/amex-hack-2019/test_QyjYwdj.csv')
df_train

# Any results you write to the current directory are saved as output.

In [ ]:
df_train['dtype'] = 'train'
df_test['dtype'] = 'test'
df_test['redemption_status'] = 0

In [ ]:
df_train_test = pd.concat([df_train, df_test])

In [ ]:
df_train_test = df_train_test.set_index("campaign_id").join(df_camp.set_index('campaign_id')).reset_index()
df_train_test = df_train_test.set_index("customer_id").join(df_demo.set_index('customer_id')).reset_index()

In [ ]:
df_train_test

In [ ]:
features = ['campaign_type', 'age_range', 'marital_status', 'rented', 'family_size',
       'no_of_children', 'income_bracket']
label = "redemption_status"

In [ ]:
df_train_test.loc[df_train_test.no_of_children.isnull(),"no_of_children"] = 0

In [ ]:
impute_feature_cat= ['marital_status', 'age_range','rented', 'family_size', 'income_bracket', 'no_of_children']

In [ ]:
simple_imputer_cat = SimpleImputer(strategy='most_frequent')

In [ ]:
def simple_impute(df, imputer, cols, test=False):
    df = df.copy()
    df['family_size'] = df['family_size'].apply(lambda x: int(x.strip('+')) if isinstance(x, str) else x)
    df['no_of_children'] = df['no_of_children'].apply(lambda x: int(x.strip('+')) if isinstance(x, str) else x)
    if not test:
        imputer.fit(df[cols])
    df[cols] = imputer.transform(df[cols])
    return df

In [ ]:
df_train_test = simple_impute(df_train_test, simple_imputer_cat, impute_feature_cat)

In [ ]:
df_train_test

In [ ]:
orde = OrdinalEncoder()

In [ ]:
def encode(df, encoder, cols, test=False):
    df = df.copy()
    if not test:
        encoder.fit(df[cols])
    df[cols] = encoder.transform(df[cols])
    return df

In [ ]:
df_train_test = encode(df_train_test, orde, ['marital_status', 'age_range', 'campaign_type'])

In [ ]:
df_split_test  = df_train_test[df_train_test.dtype == "test"].reset_index()
df_split_train  = df_train_test[df_train_test.dtype == "train"].reset_index()

df_split_test = df_split_test.drop(columns=['redemption_status']).reset_index()

In [ ]:
df_split_train

In [ ]:
x_train, x_valid,y_train,y_valid = train_test_split(df_split_train[features],df_split_train[label], test_size=0.10, random_state=42)

In [ ]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=5, random_state=42)
clf = model.fit(x_train,y_train)

In [ ]:
print("train score:", clf.score(x_train,y_train))
y_predict = clf.predict(x_valid)
clf.score(x_valid,y_valid)

In [ ]:
df_split_test[label] = clf.predict(df_split_test[features])

In [ ]:
df_split_test[['id',label]].to_csv("submission.csv", index=False)